In [1]:
import math
import os
import subprocess
import time
import scipy
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import pandas as pd
import itertools
import pickle
from tabulate import tabulate
from g4beam import *
from tqdm import *
from skopt import gp_minimize
import os

In [2]:

print(os.getcwd())  # Print current working directory
print(os.listdir())

/Applications/G4beamline-3.08.app/Contents/MacOS
['G4_PhaseRotation.g4bl', 'g4blgui', 'g4bl', 'g4bl-setup.csh', 'G4_RFCavity.g4bl', 'g4bldata', '__pycache__', 'g4beam.py', 'g4bl-setup.sh', 'g4blmpi', 'g4bltest', 'G4_FinalCooling_auto.g4bl', 'optimization.ipynb']


In [3]:
t_emit = 0.145  # mm
momentum = 100  # MeV/c
beta = 0.03    # m
alpha = 0.7     # dimensionless
l_emit = 1      # mm
pz_std = 1    # MeV/c
vd_dist = 24    # mm

pre_w1 = gen_distribution((beta, alpha, t_emit, 0, 0), (beta, alpha, t_emit, 0, 0), momentum, pz_std, z_emit=l_emit, N=20000)
pre_w1["PDGid"] = -13
print(pre_w1)
print_all_params(pre_w1)

              x         y  z         Px         Py          Pz         t  \
0     -0.512976  2.885077  0  -3.900535  -9.029183   99.061368  0.696493   
1     -1.376636 -3.719727  0   3.539987  11.694466   99.190589  0.392911   
2      1.877653  0.016131  0 -16.113389  -3.887987   98.715362  0.187208   
3      0.876056 -0.077656  0 -12.933559   9.096587  100.127330 -0.666493   
4     -2.104772  0.322508  0   6.748295   3.697277   98.975555 -1.144864   
...         ...       ... ..        ...        ...         ...       ...   
19995  1.531184 -0.648793  0  -2.816189  -3.842835   99.345282 -1.438222   
19996 -0.490784 -2.559495  0  -4.297357  10.221110  101.233700 -0.850441   
19997 -0.261358 -1.511991  0   1.321837   0.930495   99.840251  0.684917   
19998 -1.307699  0.332062  0   6.870049  -7.941351   98.390256  0.525900   
19999  3.187979  0.731205  0  -3.078000   0.219092  100.336890  0.008121   

       PDGid  EventID  TrackID  ...  ProperTime  PathLength PolX PolY PolZ  \
0        

In [4]:
# Function to optimize
def func(x):
    length, angle = x
    return emittances(cut_outliers(run_distribution(pre_w1, length, angle, vd_dist, axis=0)))[0]

start = time.time()
# Run optimization
optim_result = minimize(func, [7.5, 45], method="Nelder-Mead", bounds=((1, 10), (30, 70)), options=dict(fatol=1e-6))

# Get results
w1_length, w1_angle = optim_result.x
print(f"Length = {w1_length:.2f} mm\nAngle = {w1_angle:.1f} deg")
print("Time spent:", time.time()-start)

# Runs a single case with the optimal parameters
post_w1 = run_distribution(pre_w1, w1_length, w1_angle, vd_dist, axis=0)
print_all_params(post_w1)
print_all_params(cut_outliers(post_w1))

iter value
  29 4.14839e-02
Length = 7.16 mm
Angle = 46.8 deg
Time spent: 501.5166263580322
-----------------------------
Twiss parameters for X
emit  = 0.04556837929809599 mm
beta  = 0.03523602509183571 m
gamma = 190.3912841517946 1/m
alpha = -2.389274380568179
D     = 0.020102807108334193 m
D'    = -0.06385849343773609

Twiss parameters for Y
emit  = 0.15033843765100327 mm
beta  = 0.02578062136020853 m
gamma = 57.89729406570294 1/m
alpha = -0.7018747866169119
D     = -7.504364605861698e-05 m
D'    = -0.015026329685732892

Z-emittance:  6.3016973802779415 mm
Z std: 141.40941477821403 mm
p std: 7.153541481669927 MeV/c
Mean momentum: 87.87861830311446 MeV/c
-----------------------------
-----------------------------
Twiss parameters for X
emit  = 0.04148389410337437 mm
beta  = 0.03624565584937072 m
gamma = 201.27906156849141 1/m
alpha = -2.509081823156798
D     = 0.02037828322320484 m
D'    = -0.07007553483312377

Twiss parameters for Y
emit  = 0.14766626797100857 mm
beta  = 0.025643236

In [10]:
# Calculate dispersion correction
post_correct = remove_dispersion(post_w1)

# Ignore transverse momentums
no_transverse = remove_transverse(post_correct)
print_all_params(no_transverse)

# Reverse transverse momentums in saved copy
reverse_transverse = post_correct.copy(deep=True)
reverse_transverse["Px"] *= -1
reverse_transverse["Py"] *= -1

-----------------------------
Twiss parameters for X
emit  = 0.0 mm
beta  = inf m
gamma = nan 1/m
alpha = nan
D     = -0.00015672155075714277 m
D'    = 0.0

Twiss parameters for Y
emit  = 0.0 mm
beta  = inf m
gamma = nan 1/m
alpha = nan
D     = 5.96526664345801e-05 m
D'    = 0.0

Z-emittance:  6.222899694634878 mm
Z std: 140.50339905941556 mm
p std: 7.374529188346014 MeV/c
Mean momentum: 86.96190917499999 MeV/c
-----------------------------


/Applications/G4beamline-3.08.app/Contents/MacOS/g4beam.py:112: RuntimeWarning: divide by zero encountered in scalar divide
  eb / e / 1000,  # mm --> m
/Applications/G4beamline-3.08.app/Contents/MacOS/g4beam.py:113: RuntimeWarning: invalid value encountered in scalar divide
  ey / e * 1000,  # 1/mm --> 1/m
/Applications/G4beamline-3.08.app/Contents/MacOS/g4beam.py:114: RuntimeWarning: invalid value encountered in scalar divide
  ea / e,  # dimensionless


In [8]:
drift_length = 16000
rf_length = 5153.756925848655
drift_to_start = drift_length-rf_length/2
post_drift = recenter_t(z_prop(no_transverse, drift_to_start))


In [6]:
drift_length = 16000
rf_freq = 0.025

start = time.time()
# Function to optimize
def func(x):
    rf_phase, rf_length, rf_grad = x
    drift_to_start = drift_length-rf_length/2
    post_drift = recenter_t(z_prop(no_transverse, drift_to_start))
    post_cavity = run_g4beam(post_drift, "G4_RFCavity.g4bl", RF_length=rf_length, frfcool=rf_freq, ficool=rf_phase, Vrfcool=rf_grad, nparticles=len(no_transverse))
    pre_w2 = recombine_transverse(post_cavity, reverse_transverse)
    return np.std(p_total(cut_pz(pre_w2)))

# Run optimization
optim_result = minimize(func, [0, 4700, 7], method="Nelder-Mead", bounds=((-90, 90), (2000, 6000), (1, 10)), options=dict(fatol=1e-6))

# Get results
rf_phase, rf_length, rf_grad = optim_result.x
print(f"Phase = {rf_phase:.2f} deg\nLength = {rf_length:.0f} mm\nGradient = {rf_grad:.2f} MV/m\nFrequency = {rf_freq*1000:.1f} MHz")
print("Time spent:", time.time()-start)

# Runs a single case with the optimal parameters and add the transverse back in
drift_to_start = drift_length-rf_length/2
post_drift = recenter_t(z_prop(no_transverse, drift_to_start))
post_cavity = cut_pz(recenter_t(run_g4beam(post_drift, "G4_RFCavity.g4bl", RF_length=rf_length, frfcool=rf_freq, ficool=rf_phase, Vrfcool=rf_grad)))
pre_w2 = recombine_transverse(post_cavity, reverse_transverse)
print_all_params(pre_w2)

iter value
 105 1.31014e+00
Phase = 0.00 deg
Length = 5454 mm
Gradient = 4.75 MV/m
Frequency = 25.0 MHz
Time spent: 5815.416825056076
-----------------------------
Twiss parameters for X
emit  = 0.037214111314554384 mm
beta  = 0.03534807723684505 m
gamma = 217.6380910575403 1/m
alpha = 2.587100317417446
D     = 0.006314802761521368 m
D'    = -0.302487839608447

Twiss parameters for Y
emit  = 0.13799799575485952 mm
beta  = 0.023968080505426054 m
gamma = 59.43055610594645 1/m
alpha = 0.6514877997549641
D     = 0.00011386041169212715 m
D'    = 0.027658794970300687

Z-emittance:  4.534490508162905 mm
Z std: 778.7670167080685 mm
p std: 1.3245373009688513 MeV/c
Mean momentum: 86.81551843841237 MeV/c
-----------------------------
